# 977. Squares of a Sorted Array

[Link to Problem](https://leetcode.com/problems/squares-of-a-sorted-array/)

### Description

Given an integer array `nums` sorted in **non-decreasing** order, return *an array of the squares of each number sorted in non-decreasing order*.

**Example 1:**
```
Input: nums = [-4,-1,0,3,10]
Output: [0,1,9,16,100]
Explanation: After squaring, the array becomes [16,1,0,9,100].
After sorting, it becomes [0,1,9,16,100].
```

**Example 2:**
```
Input: nums = [-7,-3,2,3,11]
Output: [4,9,9,49,121]
```

**Constraints:**
- `1 <= nums.length <= 10^4`
- `-10^4 <= nums[i] <= 10^4`
- `nums` is sorted in **non-decreasing** order.

**Follow up:** Squaring each element and sorting the new array is very trivial, could you find an `O(n)` solution using a different approach?

## My Intuition

_Write down your thoughts, ideas, and insights here._

- **Observations:**
  1. The input array is already sorted, but it contains negative numbers.
  2. If `max(abs(nums[i]))` is not too large, could use index to sort
  3. Or use two sorted array to get the final result
- **Edge cases:**
  - Array with all negative numbers
- **Expected approach and complexity:**
  1. Time: O(n)
  2. Space: O(n)

In [22]:
# Index sort approach
from typing import List
from collections import defaultdict

def sortedSquares(nums: List[int]) -> List[int]:
    nums = [abs(n) for n in nums]
    
    count = defaultdict(int)
    for n in nums:
        count[n] += 1

    arr = [None]*(max(nums)+1)
    for n in nums:
        arr[n] = n

    result = []
    for n in arr:
        if n is None:
            continue    
        freq = count[n]
        result.extend([n**2]*freq)
        
    return result

In [38]:
# Two sorted array approach

def sortedSquares(nums: List[int]) -> List[int]:
    arr1, arr2 = [], []
    arr1 = [n**2 for n in nums if n < 0]
    arr2 = [n**2 for n in nums if n >= 0]
    arr1 = arr1[::-1]
    
    result = []
    i, j = 0, 0
    while i < len(arr1) and j < len(arr2):
        if arr1[i] < arr2[j]:
            result.append(arr1[i])
            i += 1
        else:
            result.append(arr2[j])
            j += 1

    if i != len(arr1):
        result.extend(arr1[i:])
    else:
        result.extend(arr2[j:])
    return result

if __name__ == '__main__':
    # Test Case 1
    nums1 = [-4,-1,0,3,10]
    print(f"Test 1 Input: {nums1}")
    print(f"Test 1 Output: {sortedSquares(nums1)}") # Expected: [0,1,9,16,100]

    # Test Case 2
    nums2 = [-7,-3,2,3,11]
    print(f"Test 2 Input: {nums2}")
    print(f"Test 2 Output: {sortedSquares(nums2)}") # Expected: [4,9,9,49,121]

    # Test Case 3
    nums3 = [-7,-3,0]
    print(f"Test 3 Input: {nums3}")
    print(f"Test 3 Output: {sortedSquares(nums3)}") # Expected: [0,9,49]

    # Test Case 4
    nums4 = [-1]
    print(f"Test 4 Input: {nums4}")
    print(f"Test 4 Output: {sortedSquares(nums4)}") # Expected: [1]

Test 1 Input: [-4, -1, 0, 3, 10]
Test 1 Output: [0, 1, 9, 16, 100]
Test 2 Input: [-7, -3, 2, 3, 11]
Test 2 Output: [4, 9, 9, 49, 121]
Test 3 Input: [-7, -3, 0]
Test 3 Output: [0, 9, 49]
Test 4 Input: [-1]
Test 4 Output: [1]


### 1\. 程式碼審查與分析

#### ✅ 你的解法一：Index sort approach (類似 Bucket Sort)

  * **優點**：你利用了 `arr[n] = n` 的方式來排序，這在數字範圍很小且密集時非常快。
  * **潛在風險**：這個解法依賴於數字的大小 (`max(nums)`)。
      * 題目限制 `nums[i]` 最大為 $10^4$，所以你的 `arr` 大小約為 10,001，這是可行的。
      * 但如果題目數字範圍變大（例如到 $10^9$），這一行 `arr = [None]*(max(nums)+1)` 就會導致 **Memory Limit Exceeded (記憶體不足)**。
      * 這不是嚴格意義上的 $O(N)$ 空間複雜度，而是 $O(K)$，其中 $K$ 是最大數值。

#### ✅ 你的解法二：Two sorted array approach (Split & Merge)

  * **優點**：這個邏輯非常清晰！你把負數和非負數分開，平方後變成兩個有序陣列，再用類似 Merge Sort 的邏輯合併。這是標準的 $O(N)$ 解法。
  * **可優化處**：
      * 你建立了 `arr1` 和 `arr2` 兩個額外的清單，並且使用了 `[::-1]` 進行反轉，這會產生額外的記憶體開銷和操作。
      * 我們可以不需要真的切分陣列，直接在原陣列上透過**指針**來操作。

-----

### 2\. 推薦解法：雙指針法 (Two Pointers)

這題最經典且效能最好的解法是**雙指針法**。

**核心概念：**
因為原陣列是排序過的（雖然有負數），所以**平方後的最大值一定會出現在陣列的「最左邊」或「最右邊」**。我們只需要比較頭尾，把較大的平方數填入結果陣列的**尾端**即可。

#### 步驟說明：

1.  建立一個與 `nums` 等長的 `result` 陣列。
2.  設定兩個指針：`left` 指向開頭，`right` 指向結尾。
3.  設定一個 `position` 指標，從 `result` 的最後一個位置開始往前填。
4.  比較 `abs(nums[left])` 和 `abs(nums[right])`：
      * 如果左邊絕對值較大，則平方後填入 `result[position]`，並將 `left` 向右移。
      * 如果右邊絕對值較大（或相等），則平方後填入 `result[position]`，並將 `right` 向左移。
5.  重複直到填滿所有位置。

-----

### 3\. 優化後的程式碼

這是基於你的邏輯優化後的版本，你可以直接複製到你的 Notebook 中測試。

```python
from typing import List

def sortedSquares(nums: List[int]) -> List[int]:
    n = len(nums)
    result = [0] * n  # 預先建立固定大小的陣列，比 append 更有效率
    
    left = 0
    right = n - 1
    position = n - 1  # 從結果陣列的最後面開始填
    
    while left <= right:
        left_val = abs(nums[left])
        right_val = abs(nums[right])
        
        if left_val > right_val:
            result[position] = left_val ** 2
            left += 1
        else:
            result[position] = right_val ** 2
            right -= 1
            
        position -= 1
        
    return result

# 測試區塊 (保留你的測試風格)
if __name__ == '__main__':
    test_cases = [
        [-4, -1, 0, 3, 10],
        [-7, -3, 2, 3, 11],
        [-7, -3, 0],
        [-1]
    ]
    
    for i, nums in enumerate(test_cases, 1):
        print(f"Test {i} Input: {nums}")
        print(f"Test {i} Output: {sortedSquares(nums)}")
        print("-" * 20)
```

-----

### 4\. 詳細說明文件

| 項目 | 說明 |
| :--- | :--- |
| **時間複雜度** | $O(N)$。我們只遍歷了一次陣列，每個元素只被處理一次。 |
| **空間複雜度** | $O(N)$。用來儲存回傳的 `result` 陣列（這在 LeetCode 通常不計入額外空間，但實際上確實佔用記憶體）。 |
| **邏輯解釋** | 利用「已排序」的特性，直接透過比較兩端絕對值來決定平方後的大小順序，省去了分拆陣列或事後排序的步驟。 |
| **為什麼比解法二好？** | 解法二需要多次遍歷（分拆、平方、反轉、合併），而這個解法只需要一次遍歷 (One Pass)，且不需要建立中間的暫存陣列 (`arr1`, `arr2`)，執行速度更快，記憶體使用更少。 |